In [1]:
# Install necessary libraries
!pip install -q gcsfs pillow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth # Authenticate your Colab environment to access your Google Cloud account.
auth.authenticate_user()

Mounted at /content/drive


In [3]:
from PIL import Image
import gcsfs
import numpy as np
import gc


def download_from_gcs(gcs_path, local_path, chunk_size=1024 * 1024):
    """
    Download a file from GCS to local storage using chunks to save RAM.
    Args:
    - gcs_path: gsutil URI of the file.
    - local_path: Local path to save the file.
    - chunk_size: Size of the chunks to download (default: 1 MB).
    """
    fs = gcsfs.GCSFileSystem()

    # Open the GCS file for reading
    with fs.open(gcs_path, 'rb') as gcs_file:
        # Open the local file for writing
        with open(local_path, 'wb') as local_file:
            while True:
                # Read the file in chunks
                chunk = gcs_file.read(chunk_size)
                if not chunk:
                    break
                # Write the chunk to the local file
                local_file.write(chunk)

    print(f"Downloaded file from {gcs_path} to {local_path} in chunks of {chunk_size // (1024 * 1024)} MB.")
    # Clear any remaining references
    del chunk, gcs_file, local_file
    gc.collect()


def preview_image_metadata_only(image_path):
    """
    Preview the dimensions and bit depth of an image without loading the full image into memory.
    Args:
    - image_path: Path to the image file.
    """
    try:
        with Image.open(image_path) as img:
            print(f"Image Path: {image_path}")
            print(f"Dimensions: {img.width} x {img.height}")
            print(f"Mode: {img.mode}")
    except ValueError as e:
        print(f"Error processing image: {e}")


def downsize_image_by_chunk(
    input_path,
    output_path,
    percentage=50,
    bit_depth="L",
    chunk_height=1024,
    output_format="TIFF",
    quality=95
):
    """
    Downsize an image by percentage, processing it in chunks sequentially to save RAM.
    Supports saving the output image in a compressed JPEG format.
    Args:
    - input_path: Path to the input image.
    - output_path: Path to save the downsized image.
    - percentage: Percentage to downsize the image (e.g., 50 for 50% reduction).
    - bit_depth: Desired bit depth ("L" for grayscale, "RGB" for color).
    - chunk_height: Height of each chunk to process sequentially.
    - output_format: Output image format ("TIFF" or "JPEG").
    - quality: Compression quality for JPEG (1-100, default is 95).
    """
    scale_factor = percentage / 100.0

    with Image.open(input_path) as img:
        width, height = img.size
        print(f"Original dimensions: {width}x{height}")

        # Calculate new dimensions
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        print(f"New dimensions: {new_width}x{new_height}")

        downsized_chunks = []

        # Process each chunk
        for top in range(0, height, chunk_height):
            # Define the chunk's bounding box
            bottom = min(top + chunk_height, height)
            box = (0, top, width, bottom)

            # Crop the chunk
            chunk = img.crop(box)

            # Normalize 16-bit values to 8-bit range if mode is I;16
            if img.mode == "I;16":
                chunk_array = np.array(chunk, dtype=np.float32)
                chunk_normalized = ((chunk_array / chunk_array.max()) * 255).astype(np.uint8)
                chunk = Image.fromarray(chunk_normalized)

            # Downsize the chunk
            resized_chunk = chunk.resize((new_width, int((bottom - top) * scale_factor)), Image.Resampling.LANCZOS)

            # Convert to desired bit depth
            downsized_chunk = resized_chunk.convert(bit_depth)

            # Store downsized chunk
            downsized_chunks.append(np.array(downsized_chunk))

            # Clear memory for the current chunk
            chunk.close()
            resized_chunk.close()
            downsized_chunk.close()
            gc.collect()

        # Combine chunks vertically
        downsized_img_array = np.vstack(downsized_chunks)

        # Convert back to PIL Image
        downsized_img = Image.fromarray(downsized_img_array)

        # Save the downsized image in the desired format
        if output_format.upper() == "JPEG":
            downsized_img = downsized_img.convert("RGB")  # JPEG requires RGB mode
            downsized_img.save(output_path, format="JPEG", quality=quality)
            print(f"Image downsized and saved as JPEG at {output_path} with quality {quality}.")
        else:
            downsized_img.save(output_path, format=output_format.upper())
            print(f"Image downsized and saved as {output_format.upper()} at {output_path}.")

        # Clear memory
        del downsized_chunks, downsized_img_array, downsized_img
        gc.collect()




In [4]:
# channels = ['DAPI' , 'PolyT', 'Cellbound1', 'Cellbound2', 'Cellbound3', 'Anti-Ms-CD45RO']
channels = ['PolyT']

for channel in channels:
  # Define file paths
  z_level = 3
  raw_path = f'gs://vz-ffpe-showcase/HumanUterineCancerPatient2-ROCostain/images/mosaic_{channel}_z{z_level}.tif'
  local_path = f'/content/drive/MyDrive/mosaic_{channel}_z{z_level}.tif'
  output_path = f'/content/drive/MyDrive/mosaic_{channel}_downsized_z{z_level}.tif'


  # Step 1: Download the file
  download_from_gcs(raw_path, local_path, chunk_size=1 * 1024 * 1024 * 1024) # Download the file in chunks of 1 GB


  # Step 2: Preview image details
  ## Increase the MAX_IMAGE_PIXELS limit
  Image.MAX_IMAGE_PIXELS = None  # Removes the limit
  preview_image_metadata_only(local_path)


  # Step 3: Downsize the image
  downsize_image_by_chunk(local_path, output_path, percentage=10, bit_depth="L", chunk_height=1024, output_format="TIFF")

Downloaded file from gs://vz-ffpe-showcase/HumanUterineCancerPatient2-ROCostain/images/mosaic_PolyT_z3.tif to /content/drive/MyDrive/mosaic_PolyT_z3.tif in chunks of 1024 MB.
Image Path: /content/drive/MyDrive/mosaic_PolyT_z3.tif
Dimensions: 98353 x 118725
Mode: I;16
Original dimensions: 98353x118725
New dimensions: 9835x11872
Image downsized and saved as TIFF at /content/drive/MyDrive/mosaic_PolyT_downsized_z3.tif.


# Print the environment set up

In [5]:
!pip list

Package                            Version
---------------------------------- ------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiohappyeyeballs                   2.4.4
aiohttp                            3.11.11
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            6.1.7
astropy-iers-data                  0.2025.1.6.0.33.42
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic                           4.1.0
attrs                              24.3.0
audioread            